In [17]:
import json
import psycopg2
import requests
from tqdm import tqdm
from dotenv import load_dotenv
import os

load_dotenv()

def embed_text(raw_text):
    headers = {'Content-Type': 'application/json'}
    payload = {"model": "bge-m3", 'prompt': raw_text}

    response = requests.post("http://192.168.68.66:11434/api/embeddings",
                             data=json.dumps(payload),
                             headers=headers)
    return response.json().get("embedding")


# use psycopg2 or psycopg2-binary to do database connection and grab all verses
# in for loop embed each verse one at a time
# in for loop save the embedding back to the row by id

In [18]:
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
embedding_api = os.getenv("EMBEDDING_API")
embedding_model = os.getenv("EMBEDDING_MODEL")

In [20]:
#connection parameters for psycopg2
connection = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

In [21]:
from psycopg2 import OperationalError

def create_connection():
    try:
        connection = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Connection to PostgreSQL successful")
        return connection
    except OperationalError as e:
        print(f"The error '{e}' occurred")
        return None

In [22]:
connection = create_connection()

Connection to PostgreSQL successful


In [11]:
# create a cursor to allow database queries
cursor = connection.cursor()

# define a SQL query to select all verses
query = "SELECT id, scripture_text FROM verses;"
# execute the query
cursor.execute(query)
# fetch all the verses
verses = cursor.fetchall()

In [13]:
# Loop through each verse, generate an embedding, and update the table
for verse in tqdm(verses, desc="Processing verses", unit="verse"):
    # grab the verse ID
    verse_id = verse[0]
    # extract the text
    scripture_text = verse[1]
    #generate an embedding for the text
    embedding = embed_text(scripture_text)
    # update query to save embedding back to the correct column
    update_query = """
    UPDATE verses
    SET embeddings = %s
    WHERE id = %s;
    """
    # execute update query
    cursor.execute(update_query, (embedding, verse_id))
# commit the transaction to save changes to the database
    connection.commit()

print("all verses have been embedded and updated!")

Processing verses: 100%|████████████████████████████████████████████████████| 41995/41995 [3:22:46<00:00,  3.45verse/s]

all verses have been embedded and updated!


In [23]:
#Close the cursor and connection
cursor.close()
connection.close()